## Create table 2

This code reads the output files from the models and compiles table 2. 

The model names in the paper map into the code according to: 

   * Benchmark:  \01Benchmark
   * Sunk Cost:   \02Sunkcost
   * Sunk-cost High: \03SunkHigh
   * No cost: \04NoCostSluggish
   * Reentry:  \05ReEntry
   * Search:  \06Search
   * Starters: \07Starter

In [1]:
import json

### Extract the data

The following functions extract the numbers needed from the tables from the model.out files. 

In [2]:
def pack_params(x, params):
    temp = [elm for elm in x.split(' ') if len(elm)!=0]
    
    while len(temp) > 0:
        v, k = temp.pop(), temp.pop()
        params[k] = v
        
    return params

In [3]:
def get_moments(file_path, params):
    fr = open(file_path)
    buffer = fr.readlines()
    buffer = [b.strip() for b in  buffer]
    buffer = [b for b in buffer if len(b)>0]
    
    kludge_counter = 0
    
    for i, elm in enumerate(buffer):
        
        if elm.startswith('Stopper rate: n1'):
            params['stopper'] = elm.split()[-1]
        
        if elm.startswith('Starter ratio'):
            if buffer[i+1].split()[-1] == '.':
                params['starter'] = '--'
            else:
                params['starter'] = str(float(buffer[i+1].split()[-1])*100)
            
        if elm.startswith('Starter Export Share'):
            if buffer[i+1].split()[-1] == '.':
                params['start_exs'] = '--'
            else:
                params['start_exs'] = str(float(buffer[i+1].split()[-1])*100)
            
        if elm.startswith("5y ago exporters' export share"):
            params['exs_5'] = elm.split()[-1]
            
        if 'lambda' in elm.split():
            pos = elm.split().index('lambda')
            params['lambda'] = str(float(buffer[i+1].split()[pos])*100)
            
        if elm.startswith('Exporter rate: nx'):
            params['ex_part'] = elm.split()[-1]
            
        if elm.startswith('Exporter Export Intensity: xi'):
            params['ex_intens'] = elm.split()[-1]
            
        if elm.startswith('Exporter Premium'):
            params['prem'] = str(float(elm.split()[-1])*100)
            
        if elm.startswith('Aveage xi'):
            params['avg_xi'] = str(float(buffer[i+1].strip())*100)
            
        # The no-cost model does not report the avg xi
        # Rather than fix the model, I added this code. I am sorry. 
        if ('NoCost' in file_path) and (elm.startswith('xiL')) and (kludge_counter == 0):
            params['avg_xi'] = str(float(elm.split()[2].replace(';',''))*100)
            kludge_counter = 1
            
    return params

In [4]:
def pack_costs(file_path, params):
    fr = open(file_path)
    buffer = fr.readlines()
    
    for i, elm in enumerate(buffer):
        if elm.strip()[:8] == 'Payments':
            start = i
            break
    
    params['cost_create'] = str(float(buffer[start+1])*100)
    params['cost_profit'] = str(float(buffer[start+4])*100)
            
    return params

In [5]:
def pack_rmse(file_path, params):
    fr = open(file_path)

    cnt = 0
    for line in fr:
        if line.strip()[:4] == 'RMSE':
            cnt = cnt + 1 
            if cnt == 3: 
                params['rmse_ex'] = line.split()[-1]
            if cnt == 4:
                params['rmse_empest'] = line.split()[-1]
                break
    return params

In [6]:
def read_params(file_path, params):
    fp =  open(file_path)
    buffer = fp.readlines()

    # Find the first line 
    for i, elm in enumerate(buffer):
        if elm.strip()[0:4]=='beta':
            start = i
        
    
    i = start
    while len(buffer[i].strip()) != 0:
        params = pack_params(buffer[i], params)
        i = i+1
    
    return params

### Read in the model parameters

Here we read in the model output files and store the calibrated parameter values in a list of dictionaries. 

In [7]:
filepaths = ['../Model code/01Benchmark/BenchamrkStats01.out',
             '../Model code/02SunkCost/SunkStats_TR01.out',
             '../Model code/03SunkHigh/Sunk_High_Stat.out',
             '../Model code/04NoCostSluggish/NoCost_Stat.out',
             '../Model code/05ReEntry/Reentry_Stat01.out',
             '../Model code/06Search/Search_Stat.out',
             '../Model code/07Starter/Starter_Stat.out']
params = [{}, {}, {}, {}, {}, {}, {}]

for f, p in zip(filepaths, params):
    p = read_params(f, p)
    p = pack_rmse(f, p)
    p = pack_costs(f, p)
    p = get_moments(f, p)

params[3]['exs_5'] = '--'
params[3]['prem'] = '100.0'

### Adjust parameters for reporting

We report some parameters as ratios. 

In [8]:
for i, elm in enumerate(params):
    
    # We express kappaE as kappaE*theta
    elm['thetafE'] = str(float(elm['theta'])*float(elm['f_E']))
    
    # We report KH/KE and KH/KL, the no cost model (3) has KL=0 and KH=0
    if i == 3:
        elm['relfHE'] = '0'
        elm['relfHL'] = '--'
    else:
        elm['relfHE'] = str(float(elm['f_0'])/float(elm['f_E'])*100)
        elm['relfHL'] = str(float(elm['f_0'])/float(elm['f_1']))
        
    # eta is 1/3 in the search model and 1 every where else
    if i == 5:
        elm['eta'] = '0.333'
    else:
        elm['eta'] = '1.0'

    # We report KR/KL KR = KH in the sunk models
    if i in (0, 1, 2, 3):
        elm['relfRL'] = elm['relfHL']
    else:
        elm['relfRL'] = float(elm['f_R']) / float(elm['f_1']) 
    
    

### Create table 2

The next function prints out a row of the table.

In [9]:
def format_row(symbol, sym_name, params, t, fmt):
    t = t + symbol + ' & '
    
    for c in range(0, 7):
        fstr = '{0:' + fmt + '}'
        if params[c][sym_name] == '--':
            t = t + '--' + ' & '
        else:
            t = t + fstr.format(float(params[c][sym_name])) + ' & '

    t = t[:-2] + '\\\ \n'
    return t

In [10]:
mc = '\multicolumn{1}{l}{'

fw = open('table_2.tex', 'w')

fw.write(r'\documentclass[12pt,leqno]{article}' + '\n')
fw.write(r'\usepackage{booktabs}' + '\n')
fw.write(r'\usepackage[left=1in,right=1in,top=1in,bottom=1in]{geometry}' + '\n')
fw.write(r'\begin{document}' + '\n')

fw.write(r'\begin{table}[tbp]' + '\n')
fw.write(r'\small \centering')
fw.write(r'\begin{tabular}{cccccccc}' + '\n')
fw.write(r'\toprule' + '\n')

fw.write(r'\multicolumn{8}{l}{\textit{Common Parameters}}\\' + '\n')
fw.write(r'& $\beta$ & $\sigma$ & $\delta$ & $\tau$ & $\gamma_0,\gamma_1$ & $\rho,\sigma_\epsilon$ & $\mu_E$ \\'  + '\n')
fw.write(r'\cmidrule{2-8}' + '\n')
fw.write(r'& {0.96} & {2.0} & {0.10} & {1.10} & 21.0, 0.02 & 0.65, 1.32 & --1.34 \\' + '\n')


fw.write(r'\multicolumn{8}{l}{\textit{Model-specific Parameters}} \\' +'\n')
fw.write(r'& {Bench-} & {Sunk} & {Sunk-cost} & {No} & {Reentry} & {Search} & {Starters}\\[-1mm]' + '\n')
fw.write(r'& {mark} & {Cost} & {High} & {Cost} &  &  &  \\' + '\n')
fw.write(r'\midrule' + '\n')


syms = [r'$\theta$', r'$\alpha$', r'$\alpha_x$', r'$\theta\kappa_E$', r'$100\times\kappa_H/\kappa_E$',
       r'$\kappa_H/\kappa_L$', r'$\xi_H$', r'$\xi_L$', r'$\rho_{\xi}$', r'$\eta$', r'$\kappa_R/\kappa_L$']
        
names = ['theta', 'alpha', 'alpm', 'thetafE', 'relfHE', 
         'relfHL', 'xiH', 'xiL', 'rhoH', 'eta', 'relfRL']

fmts = ['0.1f', '0.3f', '0.3f', '0.1f', '0.2f', '0.2f', '0.2f', '0.2f', '0.2f', '0.2f', '0.2f']

for sym, name, fmt in zip(syms, names, fmts):
    fw.write(format_row(sym, name, params, '', fmt))

fw.write(r'&&&&&&&\\' + '\n')
fw.write(r'\multicolumn{8}{l}{\textit{Overall Fit (RMSE): Size Distributions}}\\' + '\n')
fw.write(format_row(mc + 'Estab. + Empl.}', 'rmse_empest', params, '', '0.2f'))
fw.write(format_row(mc + 'Export}', 'rmse_ex', params, '', '0.1f'))

fw.write(r'&&&&&&&\\' + '\n')
fw.write(r'\multicolumn{8}{l}{\textit{Fixed Trade Costs Relative to}}\\' + '\n')
fw.write(format_row(mc + 'Plant Creation Cost}', 'cost_create', params, '', '0.1f'))
fw.write(format_row(mc + 'Export Profits}', 'cost_profit', params, '', '0.1f'))

fw.write(r'&&&&&&&\\' + '\n')
fw.write(r'\multicolumn{8}{l}{\textit{Selected moments (Data)}}\\' + '\n')

syms = [mc + 'Exit Rate (17.0)}', mc+'Starter Ratio (25.1)}', mc+'Starter Export Share (4.9)}',
       mc+'5-yr Incumbent Share}', mc+'Dom. Expenditure Share}', mc+'Export Participation}',
       mc+'Export Intensity}', mc+'Exporter Premium}',mc+ r'Average $\xi$}']

names = ['stopper', 'starter', 'start_exs', 'exs_5', 'lambda', 'ex_part', 'ex_intens', 'prem', 'avg_xi']

for sym, name, in zip(syms, names):
    fw.write(format_row(sym, name, params, '', '0.1f'))

fw.write(r'\bottomrule' + '\n')
fw.write(r'\end{tabular}' + '\n')
fw.write(r'\end{table}'+'\n')

fw.write(r'\end{document}')
fw.close()

### Save the parameters for later. 

Save the parameters to a file to use later. The sunk-iceberg model experiment was not read in initially. I am just reading them all in again. It is fast, if not pretty. 

In [11]:
filepaths = ['../Model code/01Benchmark/BenchamrkStats01.out',
             '../Model code/02SunkCost/SunkStats_TR01.out',
             '../Model code/03SunkHigh/Sunk_High_Stat.out',
             '../Model code/04NoCostSluggish/NoCost_Stat.out',
             '../Model code/05ReEntry/Reentry_Stat01.out',
             '../Model code/06Search/Search_Stat.out',
             '../Model code/07Starter/Starter_Stat.out',
             '../Model code/08SunkIceberg/Sunk_Iceberg01.out']

mod_out = ['bench', 'sunk', 'sunkhigh', 'nocost', 'reentry', 'search', 'starter', 'sunkice']
params_out = {}

for f, m in zip(filepaths, mod_out):
    params_out[m] = read_params(f, {})
    
# make them floats
for k in params_out:
    for kk in params_out[k]:
        params_out[k][kk] = float(params_out[k][kk])

fp = open('params.json','w')
json.dump(params_out, fp)
fp.close()

Done.